# Import Packages

In [1]:
import pandas as pd
import numpy as np
import random

from scipy import spatial
from scipy.spatial.distance import cdist

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [2]:
df = pd.read_csv('../data/task1/data.csv', header=None)

print(df.shape)
df.head()

(10000, 784)


,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
##### label data #####
df_labels = pd.read_csv('../data/task1/label.csv', header = None)

print(df_labels.shape)
df_labels.head()

(10000, 1)


,0
0,7
1,2
2,1
3,0
4,4


# Analysis

In [4]:
##### create class for K-means clustering #####
class KMeC:  
    
    def __init__(self, X, num_clusters, max_iter, tolerance, distance_type, optimize=False, check_sse_increment=False):
        
        self.k = num_clusters
        self.max_iter = max_iter
        self.optimize = optimize
        self.tolerance = tolerance
        self.distance_type = distance_type
        self.num_rows, self.num_columns = X.shape
        self.check_sse_increment = check_sse_increment
        
        
    # randomly initialize and returns k centroids (shape: (k, X.shape[1]))
    def initialize_centroids(self, X):
        
        centroids = np.empty([self.k, self.num_columns]) 
        centroid_indices = np.random.choice(range(self.num_rows), size=self.k, replace=False)
        
        for i in range(self.k):
            centroids[i] = X.iloc[centroid_indices[i], ] 
            
        return centroids
    
    
    # calculate and returns the distance between two vectors
    def compute_distance(self, x, centroid):
        
        if self.distance_type == 'euclidean':
             distance = spatial.distance.euclidean(x, centroid)
                
        elif self.distance_type == 'cosine':
            distance = 1 - spatial.distance.cosine(x, centroid)
            
        elif self.distance_type == 'jaccard':
            distance = 1 - spatial.distance.jaccard(x, centroid)
            
        return distance
    
    
    # find and returns the index of the closest centroid for a given vector i.e., 'x': row vector  
    def closest_centroid(self, x, centroids):

        distances = np.empty(self.k)

        for i in range(self.k):
            distances[i] = self.compute_distance(x, centroids[i])

        return np.argmin(distances)
    
    
    # returns an array of cluster indices for all the data samples
    def create_clusters(self, centroids, X):

        clusters = np.empty(self.num_rows)

        for i in range(self.num_rows):
            clusters[i] = self.closest_centroid(X.iloc[i, ], centroids)

        return clusters
    
    
    # computes and returns the new centroids of the clusters
    def update_centroids(self, clusters, X):

        centroids = np.empty((self.k, self.num_columns))

        for i in range(self.k):
            points = X[clusters == i]
            centroids[i] = np.mean(points, axis=0)

        return centroids
    
    
    # compute SSE    
    def calculate_sse(self, clusters, centroids, X):
        
        sse = 0
        
        for i in range(len(X)):
            sse += np.sqrt(sum((X.iloc[i, ].to_numpy() - centroids[int(clusters[i])]) ** 2))
        
        return sse

    
    # fit data
    def fit_predict(self, X):
        
        centroids = self.initialize_centroids(X)
        prev_sse = float('inf')
        iteration = 0
        
        for i in range(self.max_iter):  
            iteration += 1
            prev_centroids = centroids
            
            clusters = self.create_clusters(centroids, X)                   
            centroids = self.update_centroids(clusters, X)            
            
            difference = centroids - prev_centroids; sse = self.calculate_sse(clusters, centroids, X)
            
            if self.optimize:
                if (abs(prev_sse - sse) < self.tolerance) or (not difference.any()): break
            
            elif self.check_sse_increment:
                if sse > prev_sse: 
                    print('SSE Increased')
                    break
            
            else:
                pass
            
            
            prev_sse = sse

        return clusters, sse, iteration

## Q1

In [8]:
##### Euclidean K-means #####
euclidean_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='euclidean', optimize=False, check_sse_increment=False)
euclidean_clusters, euclidean_sse, euclidean_iter = euclidean_model.fit_predict(X=df)

print(f'Euclidean_SSE: {euclidean_sse}'); print(f'Euclidean_Iteration: {euclidean_iter}')

Euclidean_SSE: 15663131.076743983
Euclidean_Iteration: 100


In [9]:
##### Cosine K-means #####
cosine_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='cosine', optimize=False, check_sse_increment=False)
cosine_clusters, cosine_sse, cosine_iter = cosine_model.fit_predict(X=df)

print(f'Cosine SSE: {cosine_sse}'); print(f'Euclidean_Iteration: {cosine_iter}')

Cosine SSE: 17196240.95592378
Euclidean_Iteration: 100


In [10]:
##### Jaccard K-means #####
jaccard_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='jaccard', optimize=False, check_sse_increment=False)
jaccard_clusters, jaccard_sse, jaccard_iter = jaccard_model.fit_predict(X=df)

print(f'Jaccard SSE: {jaccard_sse}'); print(f'Euclidean_Iteration: {jaccard_iter}')

Jaccard SSE: 18375324.140570614
Euclidean_Iteration: 100


## Q2

In [31]:
##### calculate accuracy #####
print('Euclidean Accuracy : {0:4f}'.format(accuracy_score(df_labels.values, euclidean_clusters)))
print('Cosine Accuracy : {0:4f}'.format(accuracy_score(df_labels.values, cosine_clusters)))
print('Jaccard Accuracy : {0:4f}'.format(accuracy_score(df_labels.values, jaccard_clusters)))

Euclidean Accuracy : 0.202500
Cosine Accuracy : 0.111000
Jaccard Accuracy : 0.098000


## Q3

In [5]:
##### Euclidean K-means #####
euclidean_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='euclidean', optimize=True, check_sse_increment=False)
euclidean_clusters, euclidean_sse, euclidean_iter = euclidean_model.fit_predict(X=df)

print(f'Euclidean_SSE: {euclidean_sse}'); print(f'Euclidean_Iteration: {euclidean_iter}')

Euclidean_SSE: 15759870.705218125
Euclidean_Iteration: 51


In [6]:
##### Cosine K-means #####
cosine_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='cosine', optimize=True, check_sse_increment=False)
cosine_clusters, cosine_sse, cosine_iter = cosine_model.fit_predict(X=df)

print(f'Cosine SSE: {cosine_sse}'); print(f'Euclidean_Iteration: {cosine_iter}')

Cosine SSE: 17195823.55108855
Euclidean_Iteration: 100


In [7]:
##### Jaccard K-means #####
jaccard_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='jaccard', optimize=True, check_sse_increment=False)
jaccard_clusters, jaccard_sse, jaccard_iter = jaccard_model.fit_predict(X=df)

print(f'Jaccard SSE: {jaccard_sse}'); print(f'Euclidean_Iteration: {jaccard_iter}')

Jaccard SSE: 18375324.140570614
Euclidean_Iteration: 3


## Q4

In [39]:
##### Euclidean K-means #####
euclidean_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='euclidean', optimize=False, check_sse_increment=True)
euclidean_clusters, euclidean_sse, euclidean_iter = euclidean_model.fit_predict(X=df)

print(f'Euclidean_SSE: {euclidean_sse}'); print(f'Euclidean_Iteration: {euclidean_iter}')

SSE Increased
Euclidean_SSE: 15634472.310274852
Euclidean_Iteration: 65


In [40]:
##### Cosine K-means #####
cosine_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='cosine', optimize=False, check_sse_increment=True)
cosine_clusters, cosine_sse, cosine_iter = cosine_model.fit_predict(X=df)

print(f'Cosine SSE: {cosine_sse}'); print(f'Euclidean_Iteration: {cosine_iter}')

SSE Increased
Cosine SSE: 17499729.31613196
Euclidean_Iteration: 4


In [41]:
##### Jaccard K-means #####
jaccard_model = KMeC(X=df, num_clusters=10, max_iter=100, tolerance=0.005, distance_type='jaccard', optimize=False, check_sse_increment=True)
jaccard_clusters, jaccard_sse, jaccard_iter = jaccard_model.fit_predict(X=df)

print(f'Jaccard SSE: {jaccard_sse}'); print(f'Euclidean_Iteration: {jaccard_iter}')

SSE Increased
Jaccard SSE: 18375324.140570614
Euclidean_Iteration: 2
